In [1]:
import openai
import os
def get_openai_key():
    # Check if the file exists
    if not os.path.isfile('openai_key.txt'):
        # Create the file if it does not exist and write a default value or leave it blank
        with open('openai_key.txt', 'w') as file:
            file.write('')  # You could prompt the user for a key or leave it blank
    # Read the key from the file
    with open('openai_key.txt', 'r') as file:
        return file.read().strip()


# Usage
openai.api_key = get_openai_key()

In [6]:
import re

item_count = 0

def create_element(tag, class_name, placeholder, data_type, component_name, content=""):
    global item_count
    attributes = ""
    if class_name:
        attributes += f' class="{class_name}"'
    if placeholder:
        attributes += f' placeholder="{placeholder}"'
    if data_type:
        if component_name:
            attributes += f' data-type="{data_type}" draggable="true" type="button" id="galleryItem_{component_name}_{item_count}"'
        else:
            attributes += f' data-type="{data_type}" draggable="true" type="button" id="newItem_{item_count}"'
        item_count += 1
    if tag == "input":
        attributes += ' style="border: none;box-shadow: none;text-align: center;"'
    return f"<{tag}{attributes}>{content}</{tag}>"

def append_elements_from_compact_rep(compact_string, component_name, is_child=False):
    regex = re.compile(r"(\w+)\[([^\]]+)\]|\w+\s*{")
    html = ""
    for match in regex.finditer(compact_string):
        matched_string, type, content = match.group(0), match.group(1), match.group(2)
        if type and content:
            if type in ["title", "label"]:
                div_class = "btn-primary btn-tutor-title" if type == "title" else "btn-success btn-label row"
                data_type = "page-items" if type == "title" else "page-row"
                content_html = (create_element("p", "page-item", None, None, component_name, content) +
                                create_element("p", "removeFromDom", None, None, component_name, "Tutor Title" if type == "title" else "Label"))
                html += create_element("div", f"btn {div_class} rounded-button", None, data_type, component_name, content_html)
            elif type == "input":
                input_html = create_element("input", "form-control", content, None, component_name)
                html += create_element("div", "btn btn-light btn-input-box-t rounded-button", None, "page-row", component_name, input_html)
        elif matched_string.strip().endswith("{"):
            element_type = matched_string.strip().replace("{", "").strip()
            div_type = "page-row" if element_type == "row" else "page-items"
            div_class = "btn-warning btn-row btn-row-item grid" if element_type == "row" else "btn-info btn-column"
            p_remove = create_element("p", "removeFromDom", None, None, component_name, element_type.capitalize())
            close_index = find_closing_index(compact_string, match.end())
            inner_content = compact_string[match.end():close_index]
            ul_content = append_elements_from_compact_rep(inner_content, component_name, True)
            ul_class = "list one page-item-ul grid grid-flow-col gap-1" if element_type == "row" else "list one d-flex flex-column gap-2 page-item-rl-row"
            ul = f'<ul class="{ul_class}" data-type="page-row">{ul_content}</ul>'
            html += create_element("div", f"btn {div_class} drop-box rounded-button", None, div_type, component_name, p_remove + ul)
    if is_child:
        return html
    else:
        page_div = create_element("div", None, None, "page-items", component_name, html)
        final_container_div = create_element("div", "container mx-auto flex flex-col py-1 justify-center items-center", None, None, component_name, page_div)
        return final_container_div

def build_html_from_compact_representation(compact_rep, component_name):
    return append_elements_from_compact_rep(compact_rep, component_name)



In [2]:
detailed_description = 'a tutor for solving three rational and one square root equation'
# Introduction to the task
prompt_system = "Create a dynamic and interactive tutor interface layout specifically crafted for problem-solving exercises, which commences with the precise definition of variables that are essential for the problem at hand. Each interface is to facilitate a clear, step-by-step resolution pathway, commencing with an explicit problem statement and progressing through a structured series of steps. Each step is to include text inputs that are directly linked to data sources, ensuring their correctness can be validated through an HTN (Hierarchical Task Network) process. This setup should lead to the revealing of the final solution, all while adhering to pedagogical best practices that logically sequence problem-solving elements and enforce the understanding of the educational content."

prompt_introduction = """Generate a compact representation layout string for a tutor interface based on a specific format. This format includes elements such as titles, rows, columns, labels, and inputs."""
# Explanation of the format
prompt_format = "The format uses: title[Title], row{{...}}, column{{...}} with each element stacked vertically over the other within the column, label[Label], and input[Placeholder]. Elements within rows and columns are enclosed in curly braces {{}}, and attributes are in square brackets []."
# Design instructions for the layout
prompt_design_instructions = """Design principles require that each input element be separate, for example, in an equation, there should be one input element per digit. A single equation must be on a single row. Elements within a row are arranged horizontally. Rows cannot be directly nested within other rows, nor can columns be nested within other columns. Ensure each input element is separate, such as digits in an equation, and a single equation appears on a single line. There are no interactive buttons like 'Click to solve' within the layout."""
# Instruction to transform the description into the layout string
prompt_task = """Transform the detailed description into the compact representation layout string according to the instructions."""
prompt_examples = "Example 0: A fraction is always of the form column{input[Numerator], label[____], input[Denominator]} and is never row{input[Numerator], label[/], input[Denominator]}"
prompt_examples += "Example 1: Equation row with two fraction members should be represented as a row with multiple columns inside for the members. Each column contains stacked numerator, operand (division symbol), and denominator. Include labels for operators in the main row. For a fraction equation like 1/2 + 3/4, the layout should be row{column{input[Numerator], label[____], input[Denominator]}, label[+], column{input[Numerator], label[____], input[Denominator]}, label[=], input[Result]}. This format ensures that each part of the fraction equation is clearly defined and separated for input. \
"
# Examples of the prompt format
prompt_examples += ", Example 2: A two equations solver should be represented as title[2 Equation Solver], column{label[First Equation], row{input[First Equation Coefficient 1], label[x], label[+], input[First Equation Coefficient 2], label[=], input[First Equation Result]}, label[Second Equation], row{input[Second Equation Coefficient 1], label[x], label[+], input[Second Equation Coefficient 2], label[y], label[=], input[Second Equation Result]}}."
prompt_examples += ", Example 3: For a radicals tutor interface, the representation would be title[Radicals Tutor], column{label[Solve the following radicals multiplication problem below], row{label[What is √2 * √2], input[Result]}, input{label[Simplify the expression]}}"
prompt_examples += ", Example 4: For a Squared Tutor interface, the representation would be title[Squared Tutor], column{label[Enter the number you wish to square], row{input[Number], label[=], input[Result]}}. This configuration places the label and input for the number and the result all in a single horizontal row, maintaining a clear and concise layout."
prompt_examples += ", Example 5: For a Tutor for missionaries and cannibals, the representation would be title[Missionaries and Cannibals Tutor], column{label[Instructions], row{label[Enter the number of missionaries], input[Missionaries]}, row{label[Enter the number of cannibals], input[Cannibals]}, label[Solution], row{input[First Move], label[->], input[Second Move]}, row{input[Third Move], label[->], input[Fourth Move]}}. As in This case, make sure to scaffold all the resolution steps."
prompt_examples += ", Example 6: For a tutor for calculating proper drug dosage levels for a nurse to administer to a patient, the representation would be title[Drug Dosage Tutor], column{label[Calculation Instructions], row{label[Enter the weight of the patient (in kg)], input[Patient Weight]}, row{label[Enter the recommended dosage per kg (in mg)], input[Dosage per kg]}, label[Calculated Dosage], row{input[Dosage Total], label[mg]}}"
prompt_examples += ", Example 7: For A tutor for optimizing a business workflow, the representation would be title[Business Workflow Optimization Tutor], column{label[Instructions], row{label[Enter the number of employees], input[Number of Employees]}, row{label[Enter the number of hours worked by each employee per week], input[Hours Worked]}, row{label[Enter the average number of tasks completed by an employee per hour], input[Average Tasks Completed]}, label[Optimization Solution], row{input[Current Workflow Efficiency], label[%]}, row{label[Enter the changes you wish to make], input[Suggested Changes]}, label[Calculated Optimized Efficiency], row{input[Optimized Workflow Efficiency], label[%]}}"
prompt_examples += ", Example 8: For an English article selection tutor, the representation would be title[English Article Selection Tutor], column{label[Instructions], label[Select the most appropriate article ('a', 'an', 'the' or 'no article') for each blank in the sentences below], row{label[Sentence 1], input[Sentence 1 Article Choice]}, row{label[Sentence 2], input[Sentence 2 Article Choice]}, row{label[Sentence 3], input[Sentence 3 Article Choice]}}"
prompt_examples += ", Example 9: For a tutor for conducting a cash flow analysis of a business, the representation would be title[Cash Flow Analysis Tutor], column{label[Instructions], label[Enter the relevant business data in the fields below to begin your cash flow analysis conversion], row{label[Enter the total revenue of the business for the chosen period], input[Total Revenue]}, row{label[Enter the total cost of goods sold (COGS) during the same period], input[Cost of Goods Sold]}, label[Gross Profit], row{input[Gross Profit], label[]}, row{label[Enter the total operating expenses during the same period], input[Operating Expenses]}, label[Net Profit], row{input[Net Profit], label[]}, row{label[Enter the total cash flows from investing activities (e.g. purchase of assets, investments)], input[Investing Cash Flows]}, row{label[Enter the total cash flows from financing activities (e.g. loans, dividends, repayable)], input[Financing Cash Flows]}, label[Net Cash Flow], row{input[Net Cash Flow], label[]}}"
prompt_examples += ", Example 9: For a tutor for a three members rational equation, the representation would be title[Rational Equations], column{label[Solve the rational equation], row{column{input[Numerator 1], label[___], input[Denominator 1]}, label[+], column{input[Numerator 2], label[___], input[Denominator 2]}, label[=], column{input[Numerator 3], label[___], input[Denominator 3]}}}"


# Combining all the parts to form the complete prompt
prompt = f"{prompt_system}\n{prompt_introduction}\n\n{prompt_format}\n\n{prompt_design_instructions}\n\n{prompt_examples}\n\n{prompt_task}"
instruction = f"\nDetailed Description:\n{detailed_description}\n"

response = openai.chat.completions.create(
  model="gpt-4", # Adjusted to use GPT-4
messages=[{"role": "system", "content": prompt}, {"role": "user", "content": instruction }]
)
generated_text = response.choices[0].message.content.strip()
print(generated_text)
# Return the high-level steps generated by OpenAI


Compact Representation Layout String:
title[Rational and Square Root Equations Tutor], column{label[Solve the Rational Equations], row{column{input[Numerator 1], label[___], input[Denominator 1]}, label[+], column{input[Numerator 2], label[___], input[Denominator 2]}, label[=], column{input[Numerator 3], label[___], input[Denominator 3]}}, label[Solve the Square Root Equation], row{label[√], input[Radicand], label[=], input[Result]}}


In [15]:
html_tutor = build_html_from_compact_representation(generated_text,None)

In [29]:
from bs4 import BeautifulSoup
import re

def html_to_compact_representation(html):
    soup = BeautifulSoup(html, 'html.parser')
    def process_element(element):
        tag = element.name
        if not tag:
            return ""
        class_name = element.get('class', [])
        class_name = ' '.join(class_name)
        data_type = element.get('data-type', None)
        draggable = element.get('draggable', None)
        item_id = element.get('id', None)
        text_content = element.text.strip()
        print(class_name)
        compact_rep = ""
        if tag == 'div' and 'btn' in class_name:
            if 'btn-primary' in class_name:
                compact_rep = f"title[{text_content}]"
            elif 'btn-success' in class_name:
                compact_rep = f"label[{text_content}]"
            elif 'btn-warning' in class_name or 'btn-info' in class_name:
                element_type = "row" if "btn-row" in class_name else "column"
                children = element.find_all(recursive=False)
                children_rep = ''.join(process_element(child) for child in children)
                compact_rep = f"{element_type} {{{children_rep}}}"
        elif tag == 'input' and 'form-control' in class_name:
            placeholder = element.get('placeholder', '')
            compact_rep = f"input[{placeholder}]"
        elif tag == 'ul':
            children = element.find_all(recursive=False)
            children_rep = ''.join(process_element(child) for child in children)
            compact_rep = children_rep
        elif tag == 'li':
            children = element.find_all(recursive=False)
            children_rep = ''.join(process_element(child) for child in children)
            compact_rep = children_rep
        elif tag == 'p' and 'page-item' in class_name:
            if 'removeFromDom' in class_name:
                # These are likely not to be represented in compact form, or could be a special marker
                pass
            else:
                compact_rep = text_content
        return compact_rep
    
    return process_element(soup)




In [30]:
def html_to_compact(html):
    # Use a regex to parse relevant HTML parts
    result = ""
    # This regex matches div elements with specific class and data-type attributes
    pattern = re.compile(r'<div.*?class="([^"]*).*?" data-type="([^"]*)".*?>(.*?)<\/div>', re.DOTALL)
    
    for div in re.finditer(pattern, html):
        class_info, data_type, content = div.groups()
        inner_text = re.sub(r'<[^>]+>', '', content).strip()  # Strip all HTML tags to get the text

        if 'btn-tutor-title' in class_info:
            result += f"title[{inner_text}]"
        elif 'btn-label' in class_info:
            result += f"label[{inner_text}]"
        elif 'btn-input-box-t' in class_info:
            # Find placeholder in input element
            input_match = re.search(r'<input.*?placeholder="([^"]*)".*?>', content)
            if input_match:
                placeholder = input_match.group(1)
                result += f"input[{placeholder}]"
        elif 'btn-row' in class_info or 'btn-column' in class_info:
            element_type = 'row' if 'btn-row' in class_info else 'column'
            # Recursively process the content of div for nested structure
            nested_result = html_to_compact(content)
            result += f"{element_type}{{{nested_result}}}"

    return result


In [31]:
compact_representation = html_to_compact(html_tutor)
print(compact_representation)

column{}row{}label[___Label]input[Denominator 1]input[Numerator 1]label[___Label]input[Denominator 1]label[+Label]column{}label[___Label]input[Denominator 2]input[Numerator 2]label[___Label]input[Denominator 2]label[=Label]column{}label[___Label]input[Denominator 3]input[Numerator 3]label[___Label]input[Denominator 3]column{}label[___Label]input[Denominator 1]input[Numerator 1]label[___Label]input[Denominator 1]label[+Label]column{}label[___Label]input[Denominator 2]input[Numerator 2]label[___Label]input[Denominator 2]label[=Label]column{}label[___Label]input[Denominator 3]input[Numerator 3]label[___Label]input[Denominator 3]label[Solve the Square Root EquationLabel]row{}input[Radicand]label[=Label]input[Result]label[√Label]input[Radicand]label[=Label]input[Result]label[Solve the Rational EquationsLabel]row{}label[___Label]input[Denominator 1]input[Numerator 1]label[___Label]input[Denominator 1]label[+Label]column{}label[___Label]input[Denominator 2]input[Numerator 2]label[___Label]inp

In [24]:
html_to_compact_representation(html_tutor)

''